# 3rd_2002

## Raw Data

In [2]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_3rd_governor_election(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None
):
    """
    선거 데이터를 유연하게 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # 2~4행을 헤더로, '읍면동명'이 '합계'인 행만 추출
    df = process_3rd_governor_election(
        'https://github.com/.../강원도지사선거.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계'
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 엑셀 행 번호를 파이썬 인덱스로 변환 (엑셀은 1부터, 파이썬은 0부터)
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 컬럼명 샘플 출력
    print("\n생성된 컬럼명 (처음 10개):")
    for i, col in enumerate(new_columns[:10]):
        print(f"{i}: {col}")

    # 필터링 적용
    if filter_column and filter_value:
        # 필터링할 컬럼 찾기
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

            # 필터링 적용
            filtered_df = df[df[matching_col] == filter_value].copy()

            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            print(f"'{matching_col}' == '{filter_value}'인 행만 추출")

            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
            print("필터링 없이 전체 데이터를 반환합니다.")

    return df


# 사용 예시

# 1. 강원도지사선거 - 2~4행을 헤더로, '읍면동명'이 '합 계'인 행만
# url = "https://github.com/.../강원도지사선거.xlsx"
# summary_df = process_3rd_governor_election(
#     url,
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합 계'
# )

# 2. 다른 파일 - 1~3행을 헤더로, 필터링 없이
# df_all = process_3rd_governor_election(
#     'other_file.xlsx',
#     header_rows=(1, 3)
# )

# 3. 로컬 파일 - 기본값 사용
# df = process_3rd_governor_election('강원도지사선거.xlsx')

## Seoul


In [3]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_seoul = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%84%9C%EC%9A%B8.xls'

seoul_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_seoul,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 14
데이터 행 수: 3191

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이명박
5: 민주당 김민석
6: 녹색평화당 임삼진
7: 민주노동당 이문옥
8: 사회당 원용수
9: 무소속 이경희

'투표구명' 컬럼 발견: 투표구명
필터링 전: 3191행 → 필터링 후: 25행
'투표구명' == '합계'인 행만 추출


In [4]:
seoul_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이명박,민주당 김민석,녹색평화당 임삼진,민주노동당 이문옥,사회당 원용수,무소속 이경희,_,계,투 표 율 (%),유효 투표 율 (%)
0,종로구,합계,141157,72241,37592,30353,584,1928,305,689,NaN,71451,51.2,98.9
84,중구(서울),합계,110996,56999,28220,25799,410,1167,190,516,NaN,56302,51.4,98.8
155,용산구,합계,186438,89215,47878,36435,686,2154,338,823,NaN,88314,47.9,99
249,성동구,합계,259275,122030,60188,54742,965,3274,461,1184,NaN,120814,47.1,99
360,광진구,합계,288387,128061,65409,56266,904,2873,448,1121,NaN,127021,44.4,99.2
479,동대문구,합계,293982,139856,71954,59919,1034,3398,550,1592,NaN,138447,47.6,99
607,중랑구,합계,328660,141503,72168,60936,1239,3212,529,1873,NaN,139957,43.1,98.9
741,성북구,합계,344694,159058,78071,70160,1876,4328,639,2405,NaN,157479,46.1,99
899,강북구,합계,272559,119468,56515,55030,1263,3439,461,1520,NaN,118228,43.8,99
1010,도봉구,합계,267451,124314,64873,52444,1108,3227,470,1159,NaN,123281,46.5,99.2



## Busan


In [5]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_busan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%B6%80%EC%82%B0.xls'

busan_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_busan,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1283

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 안상영
5: 민주당 한이헌
6: 민주노동당 김석준
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1283행 → 필터링 후: 16행
'투표구명' == '합계'인 행만 추출


In [6]:
busan_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 안상영,민주당 한이헌,민주노동당 김석준,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(부산),합계,45490,23688,15669,4039,3487,NaN,23195,52.1,97.9
40,서구(부산),합계,116858,47313,31336,8397,6594,NaN,46327,40.5,97.9
113,동구(부산),합계,97322,44749,29853,7808,6247,NaN,43908,46,98.1
173,영도구,합계,136484,59273,37299,12417,8545,NaN,58261,43.4,98.3
244,부산진구,합계,320729,152469,95573,28774,25497,NaN,149844,47.5,98.3
392,동래구,합계,219090,92495,59940,15482,15783,NaN,91205,42.2,98.6
481,남구(부산),합계,230494,99050,62976,17767,16906,NaN,97649,43,98.6
579,북구(부산),합계,221411,82584,49151,18109,14128,NaN,81388,37.3,98.6
667,해운대구,합계,289845,114749,69300,22401,21305,NaN,113006,39.6,98.5
779,기장군,합계,56539,32048,20143,5616,5583,NaN,31342,56.7,97.8



## Daegu


In [7]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_daegu = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%8C%80%EA%B5%AC.xls'

daegu_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_daegu,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 810

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 조해녕
5: 무소속 이재용
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 810행 → 필터링 후: 8행
'투표구명' == '합계'인 행만 추출


In [8]:
daegu_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 조해녕,무소속 이재용,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(대구),합계,70409,34423,20807,13020,NaN,33827,48.9,98.3
45,동구(대구),합계,250562,105543,66098,37617,NaN,103715,42.1,98.3
155,서구(대구),합계,206981,84479,53305,29549,NaN,82854,40.8,98.1
253,남구(대구),합계,148146,64865,31249,32866,NaN,64115,43.8,98.8
320,북구(대구),합계,294071,117993,74706,41585,NaN,116291,40.1,98.6
445,수성구,합계,320476,134338,81824,50820,NaN,132644,41.9,98.7
581,달서구,합계,414225,155487,91557,61725,NaN,153282,37.5,98.6
737,달성군,합계,109408,54866,33397,20214,NaN,53611,50.1,97.7



## Incheon


In [9]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_incheon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%9D%B8%EC%B2%9C.xls'

incheon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_incheon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 13
데이터 행 수: 814

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 안상수
5: 민주당 박상은
6: 녹색평화당 신맹순
7: 민주노동당 김창한
8: 사회당 김영규
9: _

'투표구명' 컬럼 발견: 투표구명
필터링 전: 814행 → 필터링 후: 10행
'투표구명' == '합계'인 행만 추출


In [10]:
incheon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 안상수,민주당 박상은,녹색평화당 신맹순,민주노동당 김창한,사회당 김영규,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(인천),합계,56476,27197,15627,8295,1154,987,599,NaN,26662,48.2,98
45,동구(인천),합계,57370,28452,16071,8484,971,1484,806,NaN,27816,49.6,97.8
87,남구(인천),합계,317337,124830,71928,37404,5341,5028,3276,NaN,122977,39.3,98.5
213,연수구,합계,174043,67745,40332,19621,2711,2590,1681,NaN,66935,38.9,98.8
279,남동구,합계,286397,105918,56823,33644,6905,4372,2805,NaN,104549,37,98.7
391,부평구,합계,391526,148406,78925,47685,5728,10827,3440,NaN,146605,37.9,98.8
538,계양구,합계,226315,79673,45748,25930,2376,3144,1637,NaN,78835,35.2,98.9
619,서구(인천),합계,237302,85958,46138,27657,3132,5544,2252,NaN,84723,36.2,98.6
714,강화군,합계,51913,34626,17134,14150,755,936,679,NaN,33654,66.7,97.2
776,옹진군,합계,11228,8797,5206,2340,400,322,229,NaN,8497,78.3,96.6



## Gwangju


In [11]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gwangju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B4%91%EC%A3%BC.xls'

gwangju_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gwangju,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 14
데이터 행 수: 465

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이환의
5: 민주당 박광태
6: 민주노동당 박종현
7: 무소속 정구선
8: 무소속 정동년
9: 무소속 정호선

'투표구명' 컬럼 발견: 투표구명
필터링 전: 465행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [12]:
gwangju_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이환의,민주당 박광태,민주노동당 박종현,무소속 정구선,무소속 정동년,무소속 정호선,_,계,투 표 율 (%),유효 투표 율 (%)
0,동구(광주),합계,89770,42924,6436,20363,2593,1171,9445,2144,NaN,42152,47.8,98.2
58,서구(광주),합계,204779,85482,10740,39589,5565,1778,22435,4219,NaN,84326,41.7,98.6
148,남구(광주),합계,159676,71438,7523,34966,3652,1647,18855,3679,NaN,70322,44.7,98.4
231,북구(광주),합계,324452,128528,12760,55377,9123,3568,38680,7156,NaN,126664,39.6,98.5
367,광산구,합계,175804,75449,6236,35643,8494,1713,18000,3598,NaN,73684,42.9,97.7



## Daejeon


In [13]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_daejeon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EB%8C%80%EC%A0%84.xls'

daejeon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_daejeon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 12
데이터 행 수: 440

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 염홍철
5: 자민련 홍선기
6: 무소속 김헌태
7: 무소속 정하용
8: _
9: 계

'투표구명' 컬럼 발견: 투표구명
필터링 전: 440행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [14]:
daejeon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 염홍철,자민련 홍선기,무소속 김헌태,무소속 정하용,_,계,투 표 율 (%),유효 투표 율 (%)
0,동구(대전),합계,181210,76220,32969,32035,3066,6623,NaN,74693,42.1,98
90,중구(대전),합계,195396,83818,37980,33683,3105,7742,NaN,82510,42.9,98.4
183,서구(대전),합계,333834,140783,67939,53426,4711,13131,NaN,139207,42.2,98.9
311,유성구,합계,119056,52548,24855,19643,2112,5102,NaN,51712,44.1,98.4
366,대덕구,합계,157684,64440,28089,26639,2786,5847,NaN,63361,40.9,98.3



## Ulsan


In [15]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_ulsan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%9A%B8%EC%82%B0.xls'

ulsan_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_ulsan,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 372

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 박맹우
5: 민주노동당 송철호
6: 사회당 안승천
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 372행 → 필터링 후: 5행
'투표구명' == '합계'인 행만 추출


In [16]:
ulsan_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 박맹우,민주노동당 송철호,사회당 안승천,_,계,투 표 율 (%),유효 투표 율 (%)
0,중구(울산),합계,162972,81207,45410,32569,2113,NaN,80092,49.8,98.6
77,남구(울산),합계,234702,108730,63805,40808,2996,NaN,107609,46.3,99
176,동구(울산),합계,125305,70935,28200,39071,2693,NaN,69964,56.6,98.6
235,북구(울산),합계,79943,47783,20174,25593,1424,NaN,47191,59.8,98.8
285,울주군,합계,119884,69466,40183,24505,3103,NaN,67791,57.9,97.6



## Gyeonggi


In [17]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeonggi = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EA%B8%B0.xls'

gyeonggi_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeonggi,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 3231

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 손학규
5: 민주당 진념
6: 민주노동당 김준기
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 3231행 → 필터링 후: 39행
'투표구명' == '합계'인 행만 추출


In [18]:
gyeonggi_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 손학규,민주당 진념,민주노동당 김준기,_,계,투 표 율 (%),유효 투표 율 (%)
0,수원시장안구,합계,233829,99253,59603,33422,5251,NaN,98276,42.4,99
90,수원시권선구,합계,225950,92598,55604,31178,4856,NaN,91638,41,99
180,수원시팔달구,합계,224122,88058,53550,29845,3859,NaN,87254,39.3,99.1
260,성남시수정구,합계,192566,78431,37715,34523,5176,NaN,77414,40.7,98.7
343,성남시중원구,합계,199094,76103,36501,33577,4923,NaN,75001,38.2,98.6
418,성남시분당구,합계,281390,135540,86423,44447,3846,NaN,134716,48.2,99.4
533,의정부시,합계,260157,110516,61852,40860,6429,NaN,109141,42.5,98.8
637,안양시만안구,합계,191427,78621,44315,29802,3736,NaN,77853,41.1,99
713,안양시동안구,합계,224569,104102,60990,38365,3936,NaN,103291,46.4,99.2
808,부천시원미구,합계,275354,112168,60252,45344,5443,NaN,111039,40.7,99



## Gangwon


In [19]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gangwon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B0%95%EC%9B%90.xls'

gangwon_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gangwon,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1073

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 김진선
5: 민주당 남동우
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1073행 → 필터링 후: 18행
'투표구명' == '합계'인 행만 추출


In [20]:
gangwon_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 김진선,민주당 남동우,_,계,투 표 율 (%),유효 투표 율 (%)
0,춘천시,합계,181938,95277,52497,41426,NaN,93923,52.4,98.6
120,원주시,합계,194677,97205,62405,32931,NaN,95336,49.9,98.1
240,강릉시,합계,169090,91290,72430,17571,NaN,90001,54,98.6
357,동해시,합계,74930,42530,35744,6107,NaN,41851,56.8,98.4
409,삼척시,합계,59541,41665,33307,7553,NaN,40860,70,98.1
490,태백시,합계,41864,25876,20400,5075,NaN,25475,61.8,98.5
531,정선군,합계,37031,26016,19524,5873,NaN,25397,70.3,97.6
585,속초시,합계,64193,36439,26621,9175,NaN,35796,56.8,98.2
627,고성군(강원),합계,26309,19349,14401,4490,NaN,18891,73.5,97.6
660,양양군,합계,23171,17835,13285,4093,NaN,17378,77,97.4



## Chungbuk


In [21]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_chungbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%B6%A9%EB%B6%81.xls'

chungbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_chungbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 742

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이원종
5: 자민련 구천서
6: 무소속 장한량
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 742행 → 필터링 후: 12행
'투표구명' == '합계'인 행만 추출


In [22]:
chungbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이원종,자민련 구천서,무소속 장한량,_,계,투 표 율 (%),유효 투표 율 (%)
0,청주시상당구,합계,163686,76240,46565,21875,6693,NaN,75133,46.6,98.5
67,청주시흥덕구,합계,246912,109497,66724,32496,8636,NaN,107856,44.3,98.5
173,충주시,합계,154380,83817,49029,26735,5968,NaN,81732,54.3,97.5
284,제천시,합계,105294,62841,46428,11768,3476,NaN,61672,59.7,98.1
364,단양군,합계,29210,20330,13104,5372,1310,NaN,19786,69.6,97.3
396,청원군,합계,92158,55498,28133,20965,4846,NaN,53944,60.2,97.2
467,영동군,합계,43630,31206,18517,9041,2699,NaN,30257,71.5,97
516,보은군,합계,32369,24757,8565,13877,1566,NaN,24008,76.5,97
566,옥천군,합계,45404,31654,12864,14975,2615,NaN,30454,69.7,96.2
607,음성군,합계,63645,41437,22367,14314,3562,NaN,40243,65.1,97.1



## Chungnam


In [23]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_chungnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%B6%A9%EB%82%A8.xls'

chungnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_chungnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1071

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 박태권
5: 자민련 심대평
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1071행 → 필터링 후: 15행
'투표구명' == '합계'인 행만 추출


In [24]:
chungnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 박태권,자민련 심대평,_,계,투 표 율 (%),유효 투표 율 (%)
0,천안시,합계,299432,126601,48169,74741,NaN,122910,42.3,97.1
165,공주시,합계,98876,63126,13237,48033,NaN,61270,63.8,97.1
255,보령시,합계,85349,54816,14094,39142,NaN,53236,64.2,97.1
333,아산시,합계,135721,67878,22147,43398,NaN,65545,50,96.6
424,금산군,합계,48654,31078,7740,21909,NaN,29649,63.9,95.4
471,연기군,합계,60903,36324,9793,25493,NaN,35286,59.6,97.1
517,논산시,합계,122118,60890,15016,43755,NaN,58771,49.9,96.5
601,부여군,합계,68327,46428,9130,35761,NaN,44891,67.9,96.7
669,서천군,합계,55805,37802,10536,25308,NaN,35844,67.7,94.8
721,홍성군,합계,69735,44988,15993,27266,NaN,43259,64.5,96.2



## Jeonbuk


In [25]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeonbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%84%EB%B6%81.xls'

jeonbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeonbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1151

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 라경균
5: 민주당 강현욱
6: 무소속 손주항
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1151행 → 필터링 후: 15행
'투표구명' == '합계'인 행만 추출


In [26]:
jeonbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 라경균,민주당 강현욱,무소속 손주항,_,계,투 표 율 (%),유효 투표 율 (%)
0,전주시완산구,합계,231776,100204,8191,74023,15785,NaN,97999,43.2,97.8
111,전주시덕진구,합계,197458,79884,6453,59409,12470,NaN,78332,40.5,98.1
202,군산시,합계,197267,97041,5055,79646,10182,NaN,94883,49.2,97.8
345,익산시,합계,234964,112465,9121,84169,16164,NaN,109454,47.9,97.3
475,정읍시,합계,105973,66356,4358,48685,11592,NaN,64635,62.6,97.4
571,남원시,합계,75505,52567,3673,36854,10521,NaN,51048,69.6,97.1
664,김제시,합계,87628,58806,8025,40039,8971,NaN,57035,67.1,97
748,완주군,합계,65838,41700,3306,29454,7671,NaN,40431,63.3,97
829,진안군,합계,24876,19517,1905,11812,5194,NaN,18911,78.5,96.9
866,무주군,합계,22378,17272,2063,11219,3507,NaN,16789,77.2,97.2



## Jeonnam


In [27]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeonnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%84%EB%82%A8.xls'

jeonnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeonnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 13
데이터 행 수: 1496

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 황수연
5: 민주당 박태영
6: 무소속 송재구
7: 무소속 송하성
8: 무소속 안수원
9: _

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1496행 → 필터링 후: 22행
'투표구명' == '합계'인 행만 추출


In [28]:
jeonnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 황수연,민주당 박태영,무소속 송재구,무소속 송하성,무소속 안수원,_,계,투 표 율 (%),유효 투표 율 (%)
0,목포시,합계,169428,81949,2234,48314,24124,4459,1012,NaN,80143,48.4,97.8
108,여수시,합계,224680,128464,5123,73429,32229,11837,2700,NaN,125318,57.2,97.6
275,순천시,합계,185581,102005,4062,55386,22295,13613,3792,NaN,99148,55,97.2
387,나주시,합계,80868,56762,2914,33315,9992,6442,2194,NaN,54857,70.2,96.6
467,광양시,합계,91773,54956,4620,31800,10205,4733,2097,NaN,53455,59.9,97.3
534,담양군,합계,42009,31362,1201,20837,5843,1842,768,NaN,30491,74.7,97.2
579,장성군,합계,41952,30339,1402,22468,3222,1788,717,NaN,29597,72.3,97.6
622,곡성군,합계,30366,24338,1182,14162,5489,1838,855,NaN,23526,80.1,96.7
668,구례군,합계,25437,20009,976,11588,4416,1939,536,NaN,19455,78.7,97.2
708,고흥군,합계,76860,59156,2333,23982,21414,7983,1574,NaN,57286,77,96.8



## Gyeongbuk


In [29]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeongbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EB%B6%81.xls'

gyeongbuk_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeongbuk,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 10
데이터 행 수: 1539

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 이의근
5: 무소속 조영건
6: _
7: 계
8: 투 표 율 (%)
9: 유효 투표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1539행 → 필터링 후: 24행
'투표구명' == '합계'인 행만 추출


In [30]:
gyeongbuk_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 이의근,무소속 조영건,_,계,투 표 율 (%),유효 투표 율 (%)
0,포항시북구,합계,178954,95784,82300,11615,NaN,93915,53.5,98
103,포항시남구,합계,186347,97807,82806,13093,NaN,95899,52.5,98
206,울릉군,합계,7630,6638,5598,917,NaN,6515,87,98.1
219,경주시,합계,210873,125634,107442,15084,NaN,122526,59.6,97.5
336,김천시,합계,110853,71347,59870,9741,NaN,69611,64.4,97.6
418,안동시,합계,136401,91196,74904,13742,NaN,88646,66.9,97.2
524,구미시,합계,231648,100996,83514,15633,NaN,99147,43.6,98.2
653,영주시,합계,95784,65654,55417,8555,NaN,63972,68.5,97.4
732,영천시,합계,90206,52940,41805,9466,NaN,51271,58.7,96.8
807,상주시,합계,93241,65470,53263,10269,NaN,63532,70.2,97



## Gyeongnam


In [31]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_gyeongnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EA%B2%BD%EB%82%A8.xls'

gyeongnam_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_gyeongnam,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1511

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 김혁규
5: 민주당 김두관
6: 민주노동당 임수태
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 1511행 → 필터링 후: 20행
'투표구명' == '합계'인 행만 추출


In [32]:
gyeongnam_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 김혁규,민주당 김두관,민주노동당 임수태,_,계,투 표 율 (%),유효 투표 율 (%)
0,창원시,합계,343825,166981,111414,29630,24139,NaN,165183,48.6,98.9
138,마산시,합계,316267,153345,118410,20247,12621,NaN,151278,48.5,98.7
304,진주시,합계,242187,143459,108529,23928,8679,NaN,141136,59.2,98.4
446,진해시,합계,99953,60998,47057,8519,4555,NaN,60131,61,98.6
502,통영시,합계,98533,56062,44164,7425,3347,NaN,54936,56.9,98
581,고성군(경남),합계,48212,35870,25987,5946,3085,NaN,35018,74.4,97.6
643,사천시,합계,88130,52361,40529,7984,2807,NaN,51320,59.4,98
712,김해시,합계,245332,104406,71692,25517,5753,NaN,102962,42.6,98.6
812,밀양시,합계,92677,56158,43482,7519,3947,NaN,54948,60.6,97.8
898,거제시,합계,122843,64911,44722,11535,7336,NaN,63593,52.8,98



## Jeju

In [33]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url3_jeju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/3rd_2002/%EC%A0%9C%EC%A3%BC.xls'

jeju_3rd = process_3rd_governor_election(
    file_path_or_url = blob_url3_jeju,
    header_rows = (1, 3),
    filter_column = '투표구명',
    filter_value = '합계'
)

헤더 행: 1행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 334

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 투표구명
2: 선거 인수
3: 투표수
4:         유효투표수_        후보자별 득표수_한나라당 신구범
5: 민주당 우근민
6: 민국당 신두완
7: _
8: 계
9: 투 표 율 (%)

'투표구명' 컬럼 발견: 투표구명
필터링 전: 334행 → 필터링 후: 4행
'투표구명' == '합계'인 행만 추출


In [34]:
jeju_3rd

,위원회명,투표구명,선거 인수,투표수,유효투표수_ 후보자별 득표수_한나라당 신구범,민주당 우근민,민국당 신두완,_,계,투 표 율 (%),유효 투표 율 (%)
0,제주시,합계,196356,124337,57387,62036,3402,NaN,122825,63.3,98.8
126,북제주군,합계,74211,55474,24939,27484,1734,NaN,54157,74.8,97.6
208,서귀포시,합계,60910,44602,19471,22759,1690,NaN,43920,73.2,98.5
265,남제주군,합계,56505,43066,17705,23004,1547,NaN,42256,76.2,98.1


# Batch CSV Files to ZIP

In [ ]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")